In [ ]:
import pandas as pd
import re
from openpyxl import Workbook

def month_number_to_name(month):
    """Convert month number to abbreviated month name."""
    return ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"][month - 1]

def process_paste_sheet(processed_file_path, current_month, current_year):
    # Load the processed file
    xls = pd.ExcelFile(processed_file_path)
    
    # Create a new workbook and the "Paste" sheet
    wb = Workbook()
    ws = wb.active
    ws.title = "Paste"

    # Define the headers for the "Paste" sheet
    headers = ["File Name", "Level 3", "Level 4", "Cost Grouping", "Cost Type", 
               "Finance Region", "Attribute Type", "Period", "Cost", 
               "FTE/Contractor", "Country", "Level 3.5", "Level 4.5", "Entity", 
               "Function", "Mapped Country", "MICA"]
    ws.append(headers)  # Write headers to the first row

    for sheet_name in xls.sheet_names:
        # Process sheets containing 'Actual', 'Target', or 'Fcst'
        if re.search(r'Actual|Target|Fcst', sheet_name, re.IGNORECASE):
            sheet = pd.read_excel(xls, sheet_name=sheet_name)

            # Identify cost type
            cost_type = 'Cost' if 'DC' in sheet_name else 'FTE/Contractor'

            # Handling MoM (Last 3 Months Data)
            month_columns = [col for col in sheet.columns if re.match(r'M\d{2}', col)]
            for month_col in month_columns:
                month_num = int(month_col[1:3])
                year = int(month_col[3:]) if len(month_col) > 3 else current_year

                # Check if the month falls within the last 3 months
                if ((current_month - month_num) % 12 in [0, 1, 2]) and year == current_year:
                    period_value = f"{month_number_to_name(month_num)}-{str(year)[-2:]}"
                    ws.append([sheet_name, '', '', '', '', '', 'MoM', period_value, 
                                sheet[month_col].sum() if cost_type == 'Cost' else '', 
                                sheet[month_col].sum() if cost_type == 'FTE/Contractor' else '', 
                                '', '', '', '', '', ''])

            # Handling FY Target
            if 'Target' in sheet_name:
                fy_col = [col for col in sheet.columns if f'FY {current_year}' in col]
                for col in fy_col:
                    period_value = f"{month_number_to_name(current_month)}-{str(current_year)[-2:]}"
                    ws.append([sheet_name, '', '', '', '', '', 'FY Target', period_value, 
                                sheet[col].sum() if cost_type == 'Cost' else '', 
                                sheet[col].sum() if cost_type == 'FTE/Contractor' else '', 
                                '', '', '', '', '', ''])

            # Handling FY Forecast
            if 'Fcst' in sheet_name:
                fy_col = [col for col in sheet.columns if f'FY {current_year}' in col]
                for col in fy_col:
                    period_value = f"Dec-{str(current_year)[-2:]}"
                    ws.append([sheet_name, '', '', '', '', '', 'FY Forecast', period_value, 
                                sheet[col].sum() if cost_type == 'Cost' else '', 
                                sheet[col].sum() if cost_type == 'FTE/Contractor' else '', 
                                '', '', '', '', '', ''])

            # Handling YTD (YTD Target, YTD Forecast, YTD Actual)
            ytd_total = 0
            for month_col in month_columns:
                month_num = int(month_col[1:3])
                if month_num <= current_month:
                    ytd_total += sheet[month_col].sum() if sheet[month_col].notna().any() else 0

            period_value = f"{month_number_to_name(current_month)}-{str(current_year)[-2:]}"
            if 'Target' in sheet_name:
                ws.append([sheet_name, '', '', '', '', '', 'YTD Target', period_value, 
                            ytd_total if cost_type == 'Cost' else '', 
                            ytd_total if cost_type == 'FTE/Contractor' else '', 
                            '', '', '', '', '', ''])
            elif 'Fcst' in sheet_name:
                ws.append([sheet_name, '', '', '', '', '', 'YTD Forecast', period_value, 
                            ytd_total if cost_type == 'Cost' else '', 
                            ytd_total if cost_type == 'FTE/Contractor' else '', 
                            '', '', '', '', '', ''])
            elif 'Actual' in sheet_name:
                ws.append([sheet_name, '', '', '', '', '', 'YTD Actual', period_value, 
                            ytd_total if cost_type == 'Cost' else '', 
                            ytd_total if cost_type == 'FTE/Contractor' else '', 
                            '', '', '', '', '', ''])

            # Handling Prior Year FY
            prior_year_fy_col = [col for col in sheet.columns if f'FY {current_year - 1}' in col]
            for col in prior_year_fy_col:
                period_value = f"Dec-{str(current_year - 1)[-2:]}"
                ws.append([sheet_name, '', '', '', '', '', 'Prior Year FY', period_value, 
                            sheet[col].sum() if cost_type == 'Cost' else '', 
                            sheet[col].sum() if cost_type == 'FTE/Contractor' else '', 
                            '', '', '', '', '', ''])

            # Handling Prior Year YTD
            prior_year_ytd_col = [col for col in sheet.columns if f'YTD {current_year - 1}' in col]
            for col in prior_year_ytd_col:
                period_value = f"{month_number_to_name(current_month)}-{str(current_year - 1)[-2:]}"
                ws.append([sheet_name, '', '', '', '', '', 'Prior Year YTD', period_value, 
                            sheet[col].sum() if cost_type == 'Cost' else '', 
                            sheet[col].sum() if cost_type == 'FTE/Contractor' else '', 
                            '', '', '', '', '', ''])

    # Save the final workbook
    wb.save('final_output_file.xlsx')

# Example usage
current_month = 6  # Example: June
current_year = 2024  # Example: Current year
processed_file_path = 'Processed_file.xlsx'  # Update with your file path
process_paste_sheet(processed_file_path, current_month, current_year)
